In [76]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference, BarChart3D, ProjectedPieChart, PieChart
from openpyxl.styles import Font

### Hoja pizzas a la semana

In [77]:
df_pizzas_semana = pd.read_csv('ficheros_csv/pizzas_semana.csv').rename(columns={'Unnamed: 0': 'semana'}).set_index('semana')
df_pizzas_semana.head()

,bbq_ckn,cali_ckn,ckn_alfredo,ckn_pesto,southw_ckn,thai_ckn,big_meat,classic_dlx,hawaiian,ital_cpcllo,...,spinach_supr,five_cheese,four_cheese,green_garden,ital_veggie,mediterraneo,mexicana,spin_pesto,spinach_fet,veggie_veg
semana,,,,,,,,,,,,,,,,,,,,,
0,29.0,12.5,2.5,10.5,19.0,25.0,12.0,23.0,18.0,17.0,...,13.0,18.0,14.5,6.5,7.5,7.0,19.5,6.0,8.0,11.5
1,58.0,65.0,49.0,28.5,38.5,64.0,34.0,63.5,63.5,33.5,...,27.0,60.0,64.0,24.0,27.0,19.5,37.0,28.0,43.0,38.5
2,61.5,55.0,15.5,27.0,68.5,58.5,22.0,46.0,42.0,30.5,...,26.0,54.0,47.5,13.0,22.0,19.5,47.0,29.5,45.5,45.5
3,73.5,83.0,22.0,25.0,36.5,65.5,35.0,49.0,60.0,34.5,...,18.0,58.0,68.5,20.0,36.5,25.5,34.5,20.0,37.0,42.0
4,79.0,46.0,28.5,21.5,35.0,61.5,25.0,53.5,47.0,38.0,...,17.0,42.0,54.5,14.5,31.0,22.0,57.5,25.0,34.5,32.5


In [78]:
# creamos la tabla pivote y la ponemos en un excel
tabla_pivote = df_pizzas_semana.pivot_table(index='semana')
tabla_pivote.to_excel('reportev1.xlsx', startrow=4, sheet_name='Pizzas a la semana')
wb = load_workbook('reportev1.xlsx') # este será el excel con el que trabajaremos

In [79]:

# cogemos la pestaña que nos interese
pestaña = wb['Pizzas a la semana'] # con esto identificamos la hoja

# vamos a ver de donde a donde van los datos de nuestra tabla
min_col = pestaña.min_column
max_col = pestaña.max_column
min_row = pestaña.min_row
max_row = pestaña.max_row

print(min_col, max_col, min_row, max_row)



1 33 5 58


In [80]:
# añadimos la formula media a la ultima fila de la tabla
# vamos a añadir la media al final
# para ello tendremos que llamar a cada celda por columna(letra) + fila(numero)
# ord('A') = 65, ord('Z') = 90
i = 65 # el valor que le sumamos para poner la 'A'
letra_delante = ""
letra = 66
c = 0 # indica la columna

pestaña[f"AI5"] = "Pizza"
pestaña[f"AJ5"] = "Media"

while c < 32:
    # la fila donde lo vamos a poner es la 59
    columna = letra_delante+f"{chr(letra)}"
    # Calculamos el promedio de cada fila
    # print(f"{columna}59")
    pestaña[f"AI{5+1+c}"] = pestaña[f"{columna}5"].value
    pestaña[f"AJ{5+1+c}"] = f'=SUM({columna}5:{columna}58)/53'
    #pestaña[f"{columna}59"].style = 5
    c += 1
    letra += 1
    if letra == 91:
        # cuando nos pasamos de la Z añadimos la A delante, hasta que nos volvamos a pasar and so on
        letra_delante = chr(i)
        i += 1
        letra = 65


wb.save('reportev1.xlsx')


In [81]:
# vamos a añadir un grafico de sectores con la media de las pizzas
pie = PieChart()
labels = Reference(pestaña, min_col=35, min_row=6, max_row=37)
data = Reference(pestaña, min_col=36, min_row=6, max_row=37)
print(data)
print(labels)
pie.add_data(data, titles_from_data=True)
pie.set_categories(labels)
pie.title = "Media pizzas pedidas"

# Cut the first slice out of the pie
# slice = DataPoint(idx=0, explosion=20)
# pie.series[0].data_points = [slice]

pestaña.add_chart(pie, "AF1")
pie.style = 5
wb.save('reportev2.xlsx')



'Pizzas a la semana'!$AK$6:$AK$37
'Pizzas a la semana'!$AJ$6:$AJ$37
